In [1]:
import torch
import pandas as pd
import os as os
import numpy as np

In [2]:
image_path=r"C:\Users\austi\Downloads\Flickr8k_Dataset\Flicker8k_Dataset"
captions_path=r"C:\Users\austi\Downloads\Flickr8k_text\Flickr8k.lemma.token.txt"


In [3]:
df = pd.read_csv(captions_path, sep='\t', header=None)  

In [4]:
df[['image_filename', 'caption_id']] = df[0].str.split('#', expand=True)

df = df.drop(columns=[0])

df.columns = ['caption', 'image_filename', 'caption_id']


In [5]:
df

,caption,image_filename,caption_id
0,A man in street racer armor be examine the tir...,1305564994_00513f9a5b.jpg,0
1,Two racer drive a white bike down a road .,1305564994_00513f9a5b.jpg,1
2,Two motorist be ride along on their vehicle th...,1305564994_00513f9a5b.jpg,2
3,Two person be in a small race car drive by a g...,1305564994_00513f9a5b.jpg,3
4,Two person in race uniform in a street car .,1305564994_00513f9a5b.jpg,4
...,...,...,...
40455,A girl in a pool wear goggles and surround by ...,989754491_7e53fb4586.jpg,0
40456,A girl in green goggles in a pool with three o...,989754491_7e53fb4586.jpg,1
40457,A red haired girl make a peace sign be wear ne...,989754491_7e53fb4586.jpg,2
40458,A redheaded girl offer a peace sign as she swi...,989754491_7e53fb4586.jpg,3


A man in street racer armor be examine the tire of another racer 's motorbike .
Two racer drive a white bike down a road .
Two motorist be ride along on their vehicle that be oddly design and color .
Two person be in a small race car drive by a green hill .
Two person in race uniform in a street car .


In [8]:
from torchvision import transforms
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [9]:
caption=df["caption"][0]
caption_tokens = tokenizer(caption, padding='max_length', max_length=30, truncation=True, return_tensors="pt")


In [15]:
a=caption_tokens["input_ids"].squeeze().clone().detach()

In [16]:
a

tensor([  101,  1037,  2158,  1999,  2395, 14878,  8177,  2022, 11628,  1996,
        12824,  1997,  2178, 14878,  1005,  1055,  5013,  5638,  3489,  1012,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [21]:
import torch
import pandas as pd
import os
from PIL import Image

from torchvision import transforms
from transformers import BertTokenizer
from torch.utils.data import DataLoader, Subset

class data_loader:
    def __init__(self,images_path,captions_path,tokenizer,transform=None):
        self.images_path=images_path
        self.captions_path=captions_path
        self.tokenizer=tokenizer
        self.transform=transform
        self.df = pd.read_csv(captions_path, sep='\t', header=None)  
        self.df[['image_filename', 'caption_id']] = self.df[0].str.split('#', expand=True)
        self.df = self.df.drop(columns=[0])
        self.df.columns = ['caption', 'image_filename', 'caption_id']
    def __getitem__(self,idx):
        self.image_name=self.df["image_filename"][idx]
        self.caption_id=self.df["caption_id"][idx]
        self.img_path=f"{self.images_path}\\{self.image_name}"
        self.image=Image.open(self.img_path).convert("RGB")
        self.caption=self.df["caption"][idx]
        if self.transform is not None:
            image = self.transform(self.image)

        caption_tokens = self.tokenizer(self.caption, padding='max_length', max_length=30, truncation=True, return_tensors="pt")
        caption_tensor = caption_tokens['input_ids'].squeeze() 

        return image, caption_tensor
    
def custom_collate_fn(batch):
    images, captions = zip(*batch)
    images = torch.stack(images, dim=0)
    captions = torch.nn.utils.rnn.pad_sequence(captions, batch_first=True, padding_value=0)
    return images, captions

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


images_path=r"C:\Users\austi\Downloads\Flickr8k_Dataset\Flicker8k_Dataset"
captions_path=r"C:\Users\austi\Downloads\Flickr8k_text\Flickr8k.lemma.token.txt"
dataset = data_loader(images_path=images_path, captions_path=captions_path, tokenizer=tokenizer, transform=transform)


(tensor([[[0.3451, 0.3294, 0.3529,  ..., 0.3647, 0.3529, 0.3608],
          [0.3569, 0.3059, 0.3059,  ..., 0.3608, 0.3686, 0.3804],
          [0.3843, 0.3451, 0.3725,  ..., 0.3490, 0.3804, 0.3882],
          ...,
          [0.6745, 0.6863, 0.7059,  ..., 0.7412, 0.7569, 0.7608],
          [0.7020, 0.7137, 0.7059,  ..., 0.7216, 0.7216, 0.7294],
          [0.7137, 0.7137, 0.7255,  ..., 0.7176, 0.7294, 0.7176]],
 
         [[0.3725, 0.3647, 0.3569,  ..., 0.3647, 0.3451, 0.3451],
          [0.3765, 0.3373, 0.3255,  ..., 0.3686, 0.3686, 0.3765],
          [0.3765, 0.3569, 0.3843,  ..., 0.3765, 0.3961, 0.4039],
          ...,
          [0.6706, 0.6824, 0.7020,  ..., 0.7373, 0.7529, 0.7569],
          [0.6980, 0.7098, 0.7020,  ..., 0.7176, 0.7176, 0.7255],
          [0.7098, 0.7137, 0.7216,  ..., 0.7059, 0.7255, 0.7137]],
 
         [[0.1647, 0.1490, 0.1451,  ..., 0.1647, 0.1686, 0.1804],
          [0.1843, 0.1373, 0.1294,  ..., 0.1843, 0.2039, 0.2196],
          [0.2157, 0.1608, 0.1765,  ...,